### llama2-13b-chat-hf

### Read in Data

In [ ]:
# read in data
# get the text report as input
import pandas as pd
df_text = pd.read_csv('./doc_level_gold_labels.csv')
eg = df_text
input_text = eg.text.to_list()
input_labels = eg.labels.to_list()
eg

### original prompts (4 types)

### Current prompt

In [ ]:
# # revised prompt structure
# example = """
# {"Erythema": ["redness", "redness"], "Pain in extremity": ["sore", "arm", "soreness"], "Pruritus": ["none"]}
# """
# input_template = """
# Ignore previous conversations.

# Clinical Notes: {text}

# Extract the terms mentioned in the clinical text above that indicating each of the following terms. 
# Include the terms in the output even if the terms are not explicitly mentioned in the provided report, just provide ‘none’ as the result. 
# Please follow the order of this list: {suggest} and generate output by following the requirements below:

# Requirements:
# 1. Adverse event means any symptoms or irregular test results. Therefore, procedure description, negative test results, or only the mention of the test itself are not adverse events.
# 2. If any non symptom, vague mention, or non vaccine related terms appeared in the suggested terms, just provide "none" for them in output. 
# 3. The output should be in json format like the example below shows.

# Example: 
# {example}

# Here is the JSON output:
# """

### COT Prompt

In [ ]:
# new two step prompt structure
example = """
Symptom List: {"Erythema": ["redness", "redness"], "Pain in extremity": ["sore", "arm", "soreness"], "Pruritus": ["none"]}
Suggest List: {"Erythema": ["redness", "redness"], "Pruritus": ["none"]}
"""
input_template = """
Ignore previous conversations.

Clinical Notes: {text}

First, extract an adverse event list from the clinical text above.
Adverse event means any symptoms or irregular test results. Therefore, procedure description, negative test results, or only the mention of the test itself are not adverse events.
Then, extract adverse events that indicating each of the suggested terms below from the adverse event list in previous step. 
Include the terms from the suggested list in final output even if the terms are not explicitly mentioned in the provided report, just provide ‘none’ as the result. 

Please follow the order of this list: {suggest} and the output should only include a list of symptoms and a list of matched suggest list in json format like the example below shows.

Example: 
{example}

Here is the output:
"""

### The bloke llama2 13b

In [ ]:
# Load model directly
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ",
                                             device_map = device,
                                             trust_remote_code=False,
                                             revision="main")
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ", use_fast=True)


pipe = transformers.pipeline(

    model= model, 

    tokenizer=tokenizer,

    task='text-generation',

    temperature=0.4, 

    do_sample = True,

    max_new_tokens=256,  

    repetition_penalty=1.1 

)


In [ ]:
# build up the call
# prompt_dic = {'prefix':prompt_prefix,'cloze': prompt_cloze,'heu': prompt_heu,'cot': prompt_cot}
prompt_dic = {'new_prompt': 1}
for p in prompt_dic:
    answer_lst = []
    for row in eg.iterrows():
        txt = row[1]['text']
        suggest = row[1]['labels']
        # input = input_template.format(prompt = prompt_dic[p], suggest = suggest, text = txt)
        input = input_template.format(text=txt, suggest = suggest, example = example)

        answer = pipe(input)
        answer_lst.append(answer[0]['generated_text'][len(input):].strip())
        # torch.cuda.empty_cache()

    p_col_name = p + '_llm_result'
    eg[p_col_name] = answer_lst
    result_df = eg[[p_col_name, 'final_labels', 'doc_id', 'text']]
    f_name = p + 'nothing.json'
    result_df.to_json(f_name)



In [ ]:
eg.labels.to_list()

In [ ]:
for rows in result_df.iterrows():
    print(rows[0])
    print(rows[1][0])